In [2]:
# %% codecell
import numpy as np
import Plot as plot
import math
import matplotlib.pyplot as plt
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
import plotly.offline as offline
import time
import datetime
import pickle
import os
import scipy.spatial
from scipy.spatial import distance
from sklearn.metrics.pairwise import euclidean_distances
import matplotlib as plt
import logging
from scipy import signal
import learningAlgs as classImportLA
import dataManipulation as dataMan
from itertools import permutations
import importlib
from datetime import timedelta
from multiprocessing import Pool
import multiprocessing
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.cluster import KMeans
import timeIntervalPlotter as intervalPlotter
import pysal
import warnings
import lumping_traditional as oldLumping
import boundaryFull_SS_WeightedLumping as WLumping
from importlib import reload
from scipy.stats import rayleigh
import dataManipulation as dataMan
import matplotlib.pyplot as plt
import processData as processData
import tensorflow as tf
warnings.filterwarnings('always')
# <codecell>

In [108]:
#*******************************************************************************
# %%codecell
reload(classImportLA)
dataFrame = processData.processingData()
#address is: /home/netlab/Desktop/thesis/data/1node1-3-5/
#/home/netlab/Desktop/thesis/data/500f80271400/
data = dataFrame.copy() #copying the dataFrame to have a copy of not edited data

print("hello")
# <codecell>

#*******************************************************************************
# %% codecell
data = processData.dataFrameManipulation(data)
numberOfStates = 255
reload(processData)
cuTrans = processData.markovianTransitionMatrixDegree1(data, numberOfStates, "CU")
normalizedCuTrans = processData.normalizingTransMatrix(cuTrans)
cuTrans_cpy = normalizedCuTrans.copy()

arrExtra = [0]
print("done")
def particling(x):
    if (x["col1"] - 1 not in data["col1"]) or (x["col1"] - 1 < 0):
        arrExtra[0] = x["time"]
        return 0

    elif x["timeIndex"] != data["timeIndex"][x["col1"] - 1]:
        arrExtra[0] = x["time"]
        return 0

    else:
        return ((x["time"] - arrExtra[0]).seconds)/6

data["periodParticle"] = data.apply(lambda x: particling(x), axis = 1) #numbering each 6 seconds in the dataFrame

print(cuTrans_cpy[0])
# <codecell>
#*******************************************************************************


the address of the collected data files (not alligned files or CSV files): /home/sepehr/thesis/data/500f80271400/
['500f80271400.txt']
here
500f80271400.txt is in csvChecker

we have the csv file: pulling out data

   col1                time  CU
0     0 2018-11-13 02:01:32  45
1     1 2018-11-13 02:01:38  45
2     2 2018-11-13 02:01:44  51
3     3 2018-11-13 02:01:50  53
4     4 2018-11-13 02:01:56  45
now we have the processed data from pandas
hello
please enter how long would be the chunk minutes? 10
removing weekends from the data
done
[[0.28571429 0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.0075188  0.         0.0075188
  0.         0.         0.03007519 0.         0.04511278 0.
  0.         0.09022556 0.         0.         0.02255639 0.
  0.03759398 0.         0.         0.03007519 0.         0.06015038
  0.         0.         0.07518797 0.         0.06766917 0.
  0.         0.0526

In [109]:
normalizedCuTrans_10minutes = normalizedCuTrans.copy()

steadyState_10minutes = np.zeros(shape=(normalizedCuTrans_10minutes.shape[0],
                                     normalizedCuTrans_10minutes.shape[1]))

for i in range(normalizedCuTrans_10minutes.shape[0]):
    steadyState_10minutes[i] = abs(pysal.spatial_dynamics.ergodic.
                                steady_state(normalizedCuTrans_10minutes[i]))

correlationScore_numpy = np.zeros(shape = (normalizedCuTrans_10minutes.shape[0],
                                     normalizedCuTrans_10minutes.shape[0]))

listOfNeighbours = [0 for i in range(normalizedCuTrans_10minutes.shape[0])]


for i in range(normalizedCuTrans_10minutes.shape[0]):
    for j in range(normalizedCuTrans_10minutes.shape[0]):
        input1 = (steadyState_10minutes[i] - np.mean(
            steadyState_10minutes[i])) / (np.std(steadyState_10minutes[i]) * len(
                steadyState_10minutes[i]))

        input2 = (steadyState_10minutes[j] - np.mean(steadyState_10minutes[j])
                  ) / (np.std(steadyState_10minutes[j]))
        correlation = signal.correlate(input1, input2, mode="full")
        correlationScore = correlation[int(np.floor(len(correlation)/2))]

        correlationScore_numpy[i][j] = correlationScore




In [94]:
print(correlationScore_numpy[60])

[0.9533514  0.94403192 0.9487511  0.94618745 0.9438491  0.94585643
 0.94075885 0.93317161 0.94032547 0.9298345  0.9408983  0.94265836
 0.94751072 0.94422459 0.95393276 0.95162807 0.94816113 0.95251451
 0.95238279 0.94861466 0.9521781  0.96131542 0.95611766 0.93763945
 0.9470549  0.94602549 0.92003952 0.9332129  0.92446543 0.89630882
 0.87589712 0.84696835 0.79902762 0.77467871 0.76470563 0.73509945
 0.73384352 0.77457037 0.78053818 0.7741458  0.77363427 0.76650452
 0.81899976 0.82788181 0.84884513 0.850052   0.86509306 0.86975245
 0.8843552  0.91486623 0.93017023 0.93603442 0.96553099 0.96411005
 0.97932207 0.98513439 0.98546543 0.98651505 0.98959546 0.9960696
 1.         0.99730247 0.99494425 0.99311447 0.98805981 0.97940467
 0.9728921  0.97501449 0.97743651 0.96395265 0.95553683 0.95859967]


In [110]:
i = 0
listOfNeighbours = []
takenIndexes = []
while i < correlationScore_numpy.shape[0]:
    if i in takenIndexes:
        i += 1
        continue
    
    listOfNeighbours.append([])
    listOfNeighbours[-1].append(i)
    takenIndexes.append(i)
    for j in range(i + 1, correlationScore_numpy.shape[0]):
        if (correlationScore_numpy[i][j] > 0.98) and (j not in takenIndexes):
            listOfNeighbours[-1].append(j)
            takenIndexes.append(j)
            


In [111]:
listOfNeighbours

[[0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  128,
  129,
  130,
  131,
  132,
  133,
  134,
  135,
  136,
  137,
  138,
  139,
  140,
  141,
  142,
  143],
 [40, 41, 42, 43],
 [44, 45],
 [46, 47, 48, 49, 50, 51],
 [52, 53, 54, 55, 56, 57, 58],
 [59, 60, 61, 62, 63, 64, 86, 87, 88],
 [65, 66, 67, 71, 74, 76, 77, 78, 82, 83, 84, 89],
 [68, 69, 70, 72, 73, 75, 79, 80, 81],
 [85, 90, 91, 92, 93, 94, 95, 96, 97, 98],
 [99, 100, 101, 102, 103, 104, 105],
 [106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117],
 [118, 119, 120, 121, 122, 123, 124, 125, 126, 127]]

In [112]:
data_cpy = data.copy()

In [113]:
for i in range(len(listOfNeighbours)):
    for j in range(len(listOfNeighbours[i])):
        data["timeIndex"].loc[data["timeIndex"] == listOfNeighbours[i][j]] = i


/home/sepehr/.local/lib/python3.6/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/home/sepehr/.local/lib/python3.6/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/home/sepehr/.local/lib/python3.6/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/home/sepehr/.local/lib/python3.6/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning:


A value is trying 

/home/sepehr/.local/lib/python3.6/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/home/sepehr/.local/lib/python3.6/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/home/sepehr/.local/lib/python3.6/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/home/sepehr/.local/lib/python3.6/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning:


A value is trying 

/home/sepehr/.local/lib/python3.6/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/home/sepehr/.local/lib/python3.6/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/home/sepehr/.local/lib/python3.6/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/home/sepehr/.local/lib/python3.6/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning:


A value is trying 

/home/sepehr/.local/lib/python3.6/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/home/sepehr/.local/lib/python3.6/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/home/sepehr/.local/lib/python3.6/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/home/sepehr/.local/lib/python3.6/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning:


A value is trying 

/home/sepehr/.local/lib/python3.6/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/home/sepehr/.local/lib/python3.6/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/home/sepehr/.local/lib/python3.6/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/home/sepehr/.local/lib/python3.6/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning:


A value is trying 

/home/sepehr/.local/lib/python3.6/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/home/sepehr/.local/lib/python3.6/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/home/sepehr/.local/lib/python3.6/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/home/sepehr/.local/lib/python3.6/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning:


A value is trying 

In [114]:
numberOfStates = 255
cuTrans_correlatedTimes = processData.markovianTransitionMatrixDegree1(data, numberOfStates, "CU")
normalizedCuTrans_correlatedTimes = processData.normalizingTransMatrix(cuTrans_correlatedTimes)

In [119]:
reload(oldLumping)

<module 'lumping_traditional' from '/home/sepehr/thesis/APDataML/lumping_traditional.py'>

In [120]:
for i in range(len(normalizedCuTrans_correlatedTimes)):
    percentageMatrix_list, irreducible_matrix = preparingMatrixForLumping(normalizedCuTrans_correlatedTimes[i])
    result = oldLumping.lumping(irreducible_matrix, percentageMatrix_list, False)
    nameResult = "/home/sepehr/thesis/APDataML/pickles/6normal_correlated_lumping_result2_" + str(i) + ".pickle"
    with open(nameResult, 'wb') as handle:
        pickle.dump(result, handle)
    namePercent = "/home/sepehr/thesis/APDataML/pickles/6normal_correlated_lumping_percentage2_" + str(i) + ".pickle"
    with open(namePercent, 'wb') as handle:
        pickle.dump(percentageMatrix_list, handle)

0.9999999
1.0000001
0.99999994
0.99999994
0.99999994
0.99999994
0.99999994
0.9999999
1.0000001
we have these many sectors to check:  882636
1.0000001
0.99999994
0.99999994
0.99999994
0.99999994
0.99999994
0.99999994
0.99999994
1.0000001
0.99999994
0.99999994
0.99999994
0.99999994
we have these many sectors to check:  328772
1.0000001
0.99999994
0.99999994
we have these many sectors to check:  71316
0.9999999
0.99999994
0.99999994
1.0000001
1.0000001
1.0000001
1.0000001
1.0000001
we have these many sectors to check:  584896
0.99999994
0.99999994
0.9999999
0.99999994
1.0000001
0.99999994
1.0000001
we have these many sectors to check:  740768
0.99999994
0.99999994
0.99999994
0.99999994
0.99999994
0.99999994
0.99999994
0.9999999
0.99999994
0.99999994
we have these many sectors to check:  882636
0.99999994
0.99999994
0.99999994
0.99999994
1.0000001
0.99999994
0.99999994
0.99999994
0.99999994
0.99999994
1.0000001
0.99999994
0.99999994
0.99999994
0.99999994
1.0000001
we have these many sector

In [80]:
arrayOfLumpes = []
indexesOfCandidates = [[] for i in range(len(normalizedCuTrans_correlatedTimes))]
for timeIndex in range(len(normalizedCuTrans_correlatedTimes)):
    print(timeIndex)
    name = "/home/sepehr/thesis/APDataML/pickles/4normal_correlated_lumping_result_" + str(timeIndex) + ".pickle"
    with open(name, 'rb') as handle:
        b = pickle.load(handle)
    
    if len(b) > 0:
        minDegree = np.inf        
        for i in range(len(b)):
            if b[i][0] < minDegree:
                minDegree = b[i][0]

        minError = np.inf
        for i in range(len(b)):
            if b[i][0] == minDegree and b[i][1] < minError:
                minError = b[i][1]

        for i in range(len(b)):
            if b[i][0] == minDegree and b[i][1] == minError:
                indexesOfCandidates[timeIndex].append(b[i])

name = "/home/sepehr/thesis/APDataML/4normalCorrelatedBoundaries.pickle"
with open(name, 'wb') as handle:
    pickle.dump(indexesOfCandidates, handle) 

0
[-1, 7, 24, 48, 72, 90]
[-1, 7, 24, 48, 73, 90]
[-1, 7, 25, 48, 72, 90]
[-1, 7, 25, 48, 73, 90]
[-1, 7, 25, 49, 72, 90]
[-1, 7, 25, 49, 73, 90]
[-1, 7, 26, 48, 72, 90]
[-1, 7, 26, 48, 73, 90]
[-1, 7, 26, 49, 72, 90]
[-1, 7, 26, 49, 73, 90]
[-1, 7, 26, 50, 72, 90]
[-1, 7, 26, 50, 73, 90]
[-1, 7, 26, 50, 74, 90]
[-1, 7, 26, 50, 75, 90]
[-1, 7, 27, 48, 72, 90]
[-1, 7, 27, 48, 73, 90]
[-1, 7, 27, 49, 72, 90]
[-1, 7, 27, 49, 73, 90]
[-1, 7, 27, 50, 72, 90]
[-1, 7, 27, 50, 73, 90]
[-1, 7, 27, 50, 74, 90]
[-1, 7, 27, 50, 75, 90]
[-1, 7, 27, 51, 72, 90]
[-1, 7, 27, 51, 73, 90]
[-1, 7, 27, 51, 74, 90]
[-1, 7, 27, 51, 75, 90]
[-1, 7, 28, 48, 72, 90]
[-1, 7, 28, 48, 73, 90]
[-1, 7, 28, 49, 72, 90]
[-1, 7, 28, 49, 73, 90]
[-1, 7, 28, 50, 72, 90]
[-1, 7, 28, 50, 73, 90]
[-1, 7, 28, 50, 74, 90]
[-1, 7, 28, 50, 75, 90]
[-1, 7, 28, 51, 72, 90]
[-1, 7, 28, 51, 73, 90]
[-1, 7, 28, 51, 74, 90]
[-1, 7, 28, 51, 75, 90]
[-1, 7, 28, 52, 72, 90]
[-1, 7, 28, 52, 73, 90]
[-1, 7, 28, 52, 74, 90]
[-1, 7, 28, 52

[-1, 15, 33, 57, 73, 90]
[-1, 15, 33, 57, 74, 90]
[-1, 15, 33, 57, 75, 90]
[-1, 15, 33, 57, 76, 90]
[-1, 15, 33, 57, 77, 90]
[-1, 15, 33, 57, 78, 90]
[-1, 15, 33, 57, 79, 90]
[-1, 15, 33, 57, 80, 90]
[-1, 15, 33, 57, 81, 90]
[-1, 15, 33, 57, 82, 90]
[-1, 15, 34, 48, 72, 90]
[-1, 15, 34, 48, 73, 90]
[-1, 15, 34, 49, 72, 90]
[-1, 15, 34, 49, 73, 90]
[-1, 15, 34, 50, 72, 90]
[-1, 15, 34, 50, 73, 90]
[-1, 15, 34, 50, 74, 90]
[-1, 15, 34, 50, 75, 90]
[-1, 15, 34, 51, 72, 90]
[-1, 15, 34, 51, 73, 90]
[-1, 15, 34, 51, 74, 90]
[-1, 15, 34, 51, 75, 90]
[-1, 15, 34, 52, 72, 90]
[-1, 15, 34, 52, 73, 90]
[-1, 15, 34, 52, 74, 90]
[-1, 15, 34, 52, 75, 90]
[-1, 15, 34, 52, 76, 90]
[-1, 15, 34, 53, 72, 90]
[-1, 15, 34, 53, 73, 90]
[-1, 15, 34, 53, 74, 90]
[-1, 15, 34, 53, 75, 90]
[-1, 15, 34, 53, 76, 90]
[-1, 15, 34, 53, 77, 90]
[-1, 15, 34, 54, 72, 90]
[-1, 15, 34, 54, 73, 90]
[-1, 15, 34, 54, 74, 90]
[-1, 15, 34, 54, 75, 90]
[-1, 15, 34, 54, 76, 90]
[-1, 15, 34, 54, 77, 90]
[-1, 15, 34, 54, 78, 90]


[-1, 18, 38, 60, 82, 90]
[-1, 18, 38, 60, 83, 90]
[-1, 18, 38, 60, 84, 90]
[-1, 18, 38, 61, 72, 90]
[-1, 18, 38, 61, 73, 90]
[-1, 18, 38, 61, 74, 90]
[-1, 18, 38, 61, 75, 90]
[-1, 18, 38, 61, 76, 90]
[-1, 18, 38, 61, 77, 90]
[-1, 18, 38, 61, 78, 90]
[-1, 18, 38, 61, 79, 90]
[-1, 18, 38, 61, 80, 90]
[-1, 18, 38, 61, 81, 90]
[-1, 18, 38, 61, 82, 90]
[-1, 18, 38, 61, 83, 90]
[-1, 18, 38, 61, 84, 90]
[-1, 18, 38, 61, 85, 90]
[-1, 18, 38, 61, 86, 90]
[-1, 18, 38, 62, 72, 90]
[-1, 18, 38, 62, 73, 90]
[-1, 18, 38, 62, 74, 90]
[-1, 18, 38, 62, 75, 90]
[-1, 18, 38, 62, 76, 90]
[-1, 18, 38, 62, 77, 90]
[-1, 18, 38, 62, 78, 90]
[-1, 18, 38, 62, 79, 90]
[-1, 18, 38, 62, 80, 90]
[-1, 18, 38, 62, 81, 90]
[-1, 18, 38, 62, 82, 90]
[-1, 18, 38, 62, 83, 90]
[-1, 18, 38, 62, 84, 90]
[-1, 18, 38, 62, 85, 90]
[-1, 18, 38, 62, 86, 90]
[-1, 18, 39, 50, 72, 90]
[-1, 18, 39, 50, 73, 90]
[-1, 18, 39, 50, 74, 90]
[-1, 18, 39, 50, 75, 90]
[-1, 18, 39, 51, 72, 90]
[-1, 18, 39, 51, 73, 90]
[-1, 18, 39, 51, 74, 90]


[-1, 20, 42, 56, 73, 90]
[-1, 20, 42, 56, 74, 90]
[-1, 20, 42, 56, 75, 90]
[-1, 20, 42, 56, 76, 90]
[-1, 20, 42, 56, 77, 90]
[-1, 20, 42, 56, 78, 90]
[-1, 20, 42, 56, 79, 90]
[-1, 20, 42, 56, 80, 90]
[-1, 20, 42, 57, 72, 90]
[-1, 20, 42, 57, 73, 90]
[-1, 20, 42, 57, 74, 90]
[-1, 20, 42, 57, 75, 90]
[-1, 20, 42, 57, 76, 90]
[-1, 20, 42, 57, 77, 90]
[-1, 20, 42, 57, 78, 90]
[-1, 20, 42, 57, 79, 90]
[-1, 20, 42, 57, 80, 90]
[-1, 20, 42, 57, 81, 90]
[-1, 20, 42, 57, 82, 90]
[-1, 20, 42, 58, 72, 90]
[-1, 20, 42, 58, 73, 90]
[-1, 20, 42, 58, 74, 90]
[-1, 20, 42, 58, 75, 90]
[-1, 20, 42, 58, 76, 90]
[-1, 20, 42, 58, 77, 90]
[-1, 20, 42, 58, 78, 90]
[-1, 20, 42, 58, 79, 90]
[-1, 20, 42, 58, 80, 90]
[-1, 20, 42, 58, 81, 90]
[-1, 20, 42, 58, 82, 90]
[-1, 20, 42, 59, 72, 90]
[-1, 20, 42, 59, 73, 90]
[-1, 20, 42, 59, 74, 90]
[-1, 20, 42, 59, 75, 90]
[-1, 20, 42, 59, 76, 90]
[-1, 20, 42, 59, 77, 90]
[-1, 20, 42, 59, 78, 90]
[-1, 20, 42, 59, 79, 90]
[-1, 20, 42, 59, 80, 90]
[-1, 20, 42, 59, 81, 90]


[-1, 5, 22, 46, 67, 77]
[-1, 5, 22, 46, 68, 77]
[-1, 5, 23, 46, 67, 77]
[-1, 5, 23, 46, 68, 77]
[-1, 5, 23, 47, 67, 77]
[-1, 5, 23, 47, 68, 77]
[-1, 5, 24, 46, 67, 77]
[-1, 5, 24, 46, 68, 77]
[-1, 5, 24, 47, 67, 77]
[-1, 5, 24, 47, 68, 77]
[-1, 5, 24, 48, 67, 77]
[-1, 5, 24, 48, 68, 77]
[-1, 5, 24, 48, 69, 77]
[-1, 5, 24, 48, 70, 77]
[-1, 5, 25, 46, 67, 77]
[-1, 5, 25, 46, 68, 77]
[-1, 5, 25, 47, 67, 77]
[-1, 5, 25, 47, 68, 77]
[-1, 5, 25, 48, 67, 77]
[-1, 5, 25, 48, 68, 77]
[-1, 5, 25, 48, 69, 77]
[-1, 5, 25, 48, 70, 77]
[-1, 5, 25, 49, 67, 77]
[-1, 5, 25, 49, 68, 77]
[-1, 5, 25, 49, 69, 77]
[-1, 5, 25, 49, 70, 77]
[-1, 5, 26, 46, 67, 77]
[-1, 5, 26, 46, 68, 77]
[-1, 5, 26, 47, 67, 77]
[-1, 5, 26, 47, 68, 77]
[-1, 5, 26, 48, 67, 77]
[-1, 5, 26, 48, 68, 77]
[-1, 5, 26, 48, 69, 77]
[-1, 5, 26, 48, 70, 77]
[-1, 5, 26, 49, 67, 77]
[-1, 5, 26, 49, 68, 77]
[-1, 5, 26, 49, 69, 77]
[-1, 5, 26, 49, 70, 77]
[-1, 5, 26, 50, 67, 77]
[-1, 5, 26, 50, 68, 77]
[-1, 5, 26, 50, 69, 77]
[-1, 5, 26, 50, 

...

[-1, 12, 32, 48, 72, 86]
[-1, 12, 32, 49, 71, 86]
[-1, 12, 32, 49, 72, 86]
[-1, 12, 32, 49, 73, 86]
[-1, 12, 32, 49, 74, 86]
[-1, 12, 32, 50, 71, 86]
[-1, 12, 32, 50, 72, 86]
[-1, 12, 32, 50, 73, 86]
[-1, 12, 32, 50, 74, 86]
[-1, 12, 32, 51, 71, 86]
[-1, 12, 32, 51, 72, 86]
[-1, 12, 32, 51, 73, 86]
[-1, 12, 32, 51, 74, 86]
[-1, 12, 32, 51, 75, 86]
[-1, 12, 32, 52, 71, 86]
[-1, 12, 32, 52, 72, 86]
[-1, 12, 32, 52, 73, 86]
[-1, 12, 32, 52, 74, 86]
[-1, 12, 32, 52, 75, 86]
[-1, 12, 32, 52, 76, 86]
[-1, 12, 32, 53, 71, 86]
[-1, 12, 32, 53, 72, 86]
[-1, 12, 32, 53, 73, 86]
[-1, 12, 32, 53, 74, 86]
[-1, 12, 32, 53, 75, 86]
[-1, 12, 32, 53, 76, 86]
[-1, 12, 32, 53, 77, 86]
[-1, 12, 32, 54, 71, 86]
[-1, 12, 32, 54, 72, 86]
[-1, 12, 32, 54, 73, 86]
[-1, 12, 32, 54, 74, 86]
[-1, 12, 32, 54, 75, 86]
[-1, 12, 32, 54, 76, 86]
[-1, 12, 32, 54, 77, 86]
[-1, 12, 32, 55, 71, 86]
[-1, 12, 32, 55, 72, 86]
[-1, 12, 32, 55, 73, 86]
[-1, 12, 32, 55, 74, 86]
[-1, 12, 32, 55, 75, 86]
[-1, 12, 32, 55, 76, 86]


[-1, 17, 30, 49, 71, 86]
[-1, 17, 30, 49, 72, 86]
[-1, 17, 30, 49, 73, 86]
[-1, 17, 30, 49, 74, 86]
[-1, 17, 30, 50, 71, 86]
[-1, 17, 30, 50, 72, 86]
[-1, 17, 30, 50, 73, 86]
[-1, 17, 30, 50, 74, 86]
[-1, 17, 30, 51, 71, 86]
[-1, 17, 30, 51, 72, 86]
[-1, 17, 30, 51, 73, 86]
[-1, 17, 30, 51, 74, 86]
[-1, 17, 30, 51, 75, 86]
[-1, 17, 30, 52, 71, 86]
[-1, 17, 30, 52, 72, 86]
[-1, 17, 30, 52, 73, 86]
[-1, 17, 30, 52, 74, 86]
[-1, 17, 30, 52, 75, 86]
[-1, 17, 30, 52, 76, 86]
[-1, 17, 30, 53, 71, 86]
[-1, 17, 30, 53, 72, 86]
[-1, 17, 30, 53, 73, 86]
[-1, 17, 30, 53, 74, 86]
[-1, 17, 30, 53, 75, 86]
[-1, 17, 30, 53, 76, 86]
[-1, 17, 30, 53, 77, 86]
[-1, 17, 30, 54, 71, 86]
[-1, 17, 30, 54, 72, 86]
[-1, 17, 30, 54, 73, 86]
[-1, 17, 30, 54, 74, 86]
[-1, 17, 30, 54, 75, 86]
[-1, 17, 30, 54, 76, 86]
[-1, 17, 30, 54, 77, 86]
[-1, 17, 31, 47, 71, 86]
[-1, 17, 31, 47, 72, 86]
[-1, 17, 31, 48, 71, 86]
[-1, 17, 31, 48, 72, 86]
[-1, 17, 31, 49, 71, 86]
[-1, 17, 31, 49, 72, 86]
[-1, 17, 31, 49, 73, 86]


[-1, 19, 40, 55, 74, 86]
[-1, 19, 40, 55, 75, 86]
[-1, 19, 40, 55, 76, 86]
[-1, 19, 40, 55, 77, 86]
[-1, 19, 40, 55, 78, 86]
[-1, 19, 40, 56, 71, 86]
[-1, 19, 40, 56, 72, 86]
[-1, 19, 40, 56, 73, 86]
[-1, 19, 40, 56, 74, 86]
[-1, 19, 40, 56, 75, 86]
[-1, 19, 40, 56, 76, 86]
[-1, 19, 40, 56, 77, 86]
[-1, 19, 40, 56, 78, 86]
[-1, 19, 40, 56, 79, 86]
[-1, 19, 40, 56, 80, 86]
[-1, 19, 40, 57, 71, 86]
[-1, 19, 40, 57, 72, 86]
[-1, 19, 40, 57, 73, 86]
[-1, 19, 40, 57, 74, 86]
[-1, 19, 40, 57, 75, 86]
[-1, 19, 40, 57, 76, 86]
[-1, 19, 40, 57, 77, 86]
[-1, 19, 40, 57, 78, 86]
[-1, 19, 40, 57, 79, 86]
[-1, 19, 40, 57, 80, 86]
[-1, 19, 40, 58, 71, 86]
[-1, 19, 40, 58, 72, 86]
[-1, 19, 40, 58, 73, 86]
[-1, 19, 40, 58, 74, 86]
[-1, 19, 40, 58, 75, 86]
[-1, 19, 40, 58, 76, 86]
[-1, 19, 40, 58, 77, 86]
[-1, 19, 40, 58, 78, 86]
[-1, 19, 40, 58, 79, 86]
[-1, 19, 40, 58, 80, 86]
[-1, 19, 40, 58, 81, 86]
[-1, 19, 40, 59, 71, 86]
[-1, 19, 40, 59, 72, 86]
[-1, 19, 40, 59, 73, 86]
[-1, 19, 40, 59, 74, 86]


[-1, 8, 25, 48, 73, 87]
[-1, 8, 25, 49, 70, 87]
[-1, 8, 25, 49, 71, 87]
[-1, 8, 25, 49, 72, 87]
[-1, 8, 25, 49, 73, 87]
[-1, 8, 26, 46, 70, 87]
[-1, 8, 26, 46, 71, 87]
[-1, 8, 26, 47, 70, 87]
[-1, 8, 26, 47, 71, 87]
[-1, 8, 26, 48, 70, 87]
[-1, 8, 26, 48, 71, 87]
[-1, 8, 26, 48, 72, 87]
[-1, 8, 26, 48, 73, 87]
[-1, 8, 26, 49, 70, 87]
[-1, 8, 26, 49, 71, 87]
[-1, 8, 26, 49, 72, 87]
[-1, 8, 26, 49, 73, 87]
[-1, 8, 26, 50, 70, 87]
[-1, 8, 26, 50, 71, 87]
[-1, 8, 26, 50, 72, 87]
[-1, 8, 26, 50, 73, 87]
[-1, 8, 26, 50, 74, 87]
[-1, 8, 26, 51, 70, 87]
[-1, 8, 26, 51, 71, 87]
[-1, 8, 26, 51, 72, 87]
[-1, 8, 26, 51, 73, 87]
[-1, 8, 26, 51, 74, 87]
[-1, 8, 26, 51, 75, 87]
[-1, 8, 27, 46, 70, 87]
[-1, 8, 27, 46, 71, 87]
[-1, 8, 27, 47, 70, 87]
[-1, 8, 27, 47, 71, 87]
[-1, 8, 27, 48, 70, 87]
[-1, 8, 27, 48, 71, 87]
[-1, 8, 27, 48, 72, 87]
[-1, 8, 27, 48, 73, 87]
[-1, 8, 27, 49, 70, 87]
[-1, 8, 27, 49, 71, 87]
[-1, 8, 27, 49, 72, 87]
[-1, 8, 27, 49, 73, 87]
[-1, 8, 27, 50, 70, 87]
[-1, 8, 27, 50, 

KeyboardInterrupt: 

In [79]:
indexesOfCandidates

[[], [], [], [], [], [], [], [], [], [], [], []]

In [41]:
name = "/home/sepehr/thesis/APDataML/5normalCorrelatedBoundaries.pickle"
with open(name, 'rb') as handle:
    normalLumps = pickle.load(handle)

print(normalLumps[3][0])
# for i in range(data[8][0][3]):

#****plus 1 data boundaries****************
for i in range(len(normalLumps)):
    if len(normalLumps[i]) == 0:
        continue

    for j in range(len(normalLumps[i])):
        for x in range(len(normalLumps[i][j][3])):
            normalLumps[i][j][3][x] += 1

percentageArray = []
for i in range(len(normalizedCuTrans_correlatedTimes)):
    name = "/home/sepehr/thesis/APDataML/pickles/5normal_correlated_lumping_percentage_" + str(i) + ".pickle"
    with open(name, 'rb') as handle:
        percentageArray.append(pickle.load(handle))


(1.0, 43.99560546875, array([[6.82129891e-01, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00],
       [2.80405405e-01, 3.78695209e-01, 3.32480818e-02, 6.39386189e-04,
        0.00000000e+00, 0.00000000e+00],
       [1.19047619e-01, 2.43902439e-01, 2.50000000e-01, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00],
       [8.33333333e-02, 0.00000000e+00, 7.69230769e-02, 8.33333333e-02,
        0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 1.25000000e-01, 1.25000000e-01, 1.33333333e-01,
        0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00]]), [-1, 20, 34, 52, 66, 77, 86])


In [67]:
YArraysForLearning.shape

(2152674,)